<a href="https://colab.research.google.com/github/saravanan-nj/notebooks/blob/main/qlora-tiny-llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Requirements

In [3]:
!pip install -q accelerate==1.3.0 peft==0.14.0 bitsandbytes==0.45.2 transformers==4.48.3 trl==0.14.0 datasets==3.1.0 pretty_midi # fsspec==2024.10.0

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModel,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM

In [22]:
# Load the training dataset
from google.colab import drive
import pandas as pd
""
df = pd.read_csv('/content/imdb/train_data.txt', delimiter=":::", names=["index", "title", "genre", "description"])

<ipython-input-22-7d34c7d864dd>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/imdb/train_data.txt', delimiter=":::", names=["index", "title", "genre", "description"])


In [23]:
dataset_df = pd.DataFrame()
def get_text(row):
  return f"""
### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
{row["description"]}

### Genre:
{row["genre"].strip()}
<|endresponse>
"""
print(df.head())
dataset_df["description"] = df["description"]
dataset_df["genre"] = df["genre"]
dataset_df["instruction"] = "Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie."
# dataset_df["text"] = dataset_df.apply(get_text, axis=1)

   index                               title       genre  \
0      1       Oscar et la dame rose (2009)       drama    
1      2                       Cupid (1997)    thriller    
2      3   Young, Wild and Wonderful (1980)       adult    
3      4              The Secret Sin (1915)       drama    
4      5             The Unrecovered (2007)       drama    

                                         description  
0   Listening in to a conversation between his do...  
1   A brother and sister with a past incestuous r...  
2   As the bus empties the students for their fie...  
3   To help their unemployed father make ends mee...  
4   The film's title refers not only to the un-re...  


In [24]:
from datasets import Dataset
print(dataset_df.head())
dataset = Dataset.from_pandas(dataset_df)

                                         description       genre  \
0   Listening in to a conversation between his do...      drama    
1   A brother and sister with a past incestuous r...   thriller    
2   As the bus empties the students for their fie...      adult    
3   To help their unemployed father make ends mee...      drama    
4   The film's title refers not only to the un-re...      drama    

                                         instruction  
0  Given a movie description, read the descriptio...  
1  Given a movie description, read the descriptio...  
2  Given a movie description, read the descriptio...  
3  Given a movie description, read the descriptio...  
4  Given a movie description, read the descriptio...  


In [25]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [26]:
model_name = "crumb/nano-mistral"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [27]:
device_map = "auto"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [28]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = SFTConfig(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=100,
    learning_rate=1e-5,
    weight_decay=0.01,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=5000,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    max_seq_length=256,
    packing=False
)

In [29]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))


def formatted_prompts_func(example):
  output_texts = []
  for i in range(len(example["description"])):
    output_texts.append(f"""
### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
{example["description"][i]}

### Genre:
{(example["genre"][i] or '' ).strip()}
<|endresponse>
""")
  # print_tokens_with_ids(output_texts[0])
  return output_texts

response_template ="\n### Description:"
collator = DataCollatorForCompletionOnlyLM(tokenizer.encode(response_template, add_special_tokens=False)[:2], tokenizer=tokenizer)


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
    processing_class=tokenizer,
    formatting_func=formatted_prompts_func,
    data_collator=collator
)
trainer.train()
trainer.model.save_pretrained("imdb-classifier")

Map:   0%|          | 0/54214 [00:00<?, ? examples/s]

Step,Training Loss
100,3.690800
200,3.668600
300,3.526800
400,3.396900
500,3.267100
600,3.171700
700,3.089400
800,2.998800
900,2.900600
1000,2.804200


In [33]:
!zip -r imdb-classifier.zip results imdb-classifier

  adding: results/ (stored 0%)
  adding: results/checkpoint-5000/ (stored 0%)
  adding: results/checkpoint-5000/README.md (deflated 66%)
  adding: results/checkpoint-5000/tokenizer.model (deflated 55%)
  adding: results/checkpoint-5000/special_tokens_map.json (deflated 73%)
  adding: results/checkpoint-5000/training_args.bin (deflated 51%)
  adding: results/checkpoint-5000/trainer_state.json (deflated 74%)
  adding: results/checkpoint-5000/tokenizer_config.json (deflated 69%)
  adding: results/checkpoint-5000/adapter_config.json (deflated 54%)
  adding: results/checkpoint-5000/scheduler.pt (deflated 56%)
  adding: results/checkpoint-5000/rng_state.pth (deflated 25%)
  adding: results/checkpoint-5000/adapter_model.safetensors (deflated 7%)
  adding: results/checkpoint-5000/optimizer.pt (deflated 9%)
  adding: results/checkpoint-5000/tokenizer.json (deflated 85%)
  adding: results/runs/ (stored 0%)
  adding: results/runs/Feb14_02-16-11_7c159354e9f6/ (stored 0%)
  adding: results/runs/Feb

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, '/content/imdb-classifier')
model = model.merge_and_unload()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, truncation=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
original_pipeline = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200, truncation=True)

prompt_template = """
### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
{}

### Genre:
"""

Device set to use cuda:0
Device set to use cuda:0


In [31]:
movie_description = "Müzeyyen is a young woman who lives a drug-fuelled, chaotic life outside the norms of society. Her brother Ali interrupts his own successful, bourgeois life in Bolu, and goes to Antalya to meet with his recently- discovered sister. There, the legitimate child - Ali - will be tested through unguessed - at sins, while the illegitimate one - Müzeyyen - will be tempted in turn by Ali's hope."
print(len(movie_description))
movie_input = prompt_template.format(movie_description)
result = pipe(movie_input)
original_result = original_pipeline(movie_input)
print(result[0]["generated_text"])
print("*" * 60)
print(original_result[0]["generated_text"])

390

### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
Müzeyyen is a young woman who lives a drug-fuelled, chaotic life outside the norms of society. Her brother Ali interrupts his own successful, bourgeois life in Bolu, and goes to Antalya to meet with his recently- discovered sister. There, the legitimate child - Ali - will be tested through unguessed - at sins, while the illegitimate one - Müzeyyen - will be tempted in turn by Ali's hope.

### Genre:
drama
<|endresponse>
<|endresponse>
<|endresponse>
<|endresponse>
<|endresponse>
<|endresponse>
<|end
************************************************************

### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
Müzeyyen is a young woman who lives 

In [32]:
movie_description = "Shy bookworm Tommy vacations in Palm Springs, never expecting an amorous adventure. But once he spots Brendan, he can't stop fantasizing about being with the sensuous, dark-haired man. However, Brendan's friends have their own ideas, and continue to pull him into their orgy of alcohol and sex. Finally alone at the pool, Brendan approaches Tommy and they spend an afternoon exploring the desert town and each other. Just as Tommy saves Brendan from a poolside accident, Brendan's friends come back - and try to force the two apart. True love conquers all, as Tommy's vacation climaxes in a passionate declaration of love and desire. LOVE INN EXILE boasts several unforgettable erotic scenes - including the strangely tender, drunken ménage a trois between Brendan's friends, a spicy tequila shot off Brendan's shoulder, and, of course, Tommy's ultimate fulfillment by his ideal man. LOVE INN EXILE is filmed entirely on location at Inn Exile in Palm Springs. With music by adult film star Sharon Kane, this video makes exile seem like a perfectly good punishment!"
movie_input = prompt_template.format(movie_description[:450])
result = pipe(movie_input)
original_result = original_pipeline(movie_input)
print(result[0]["generated_text"])
print("*" * 60)
print(original_result[0]["generated_text"])


### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
Shy bookworm Tommy vacations in Palm Springs, never expecting an amorous adventure. But once he spots Brendan, he can't stop fantasizing about being with the sensuous, dark-haired man. However, Brendan's friends have their own ideas, and continue to pull him into their orgy of alcohol and sex. Finally alone at the pool, Brendan approaches Tommy and they spend an afternoon exploring the desert town and each other. Just as Tommy saves Brendan from 

### Genre:
drama
<|endresponse>
  * 
  * 
  * 
  * 
  * 
  * 
  * 
  * 
  *
************************************************************

### Instruction:
Given a movie description, read the description, understand and analyse the story of the movie based on the given description and return the genre of the movie.

### Description:
Shy bookworm Tomm